In [1]:
# imports
import numpy as np
import pandas as pd
import polars as pl
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import os
import json
import seaborn as sns
from rich import print
import time

load_dotenv()

RAW_DATA_DIR = Path(os.getenv('RAW_DATA_DIR'))
PROCESSED_DATA_DIR = Path(os.getenv('PROCESSED_DATA_DIR'))

In [10]:
total_size_mb = 0
total_rows = 0

for views_file in RAW_DATA_DIR.glob('*views*.csv'):
    rows = pl.read_csv(views_file).height
    size_mb = os.stat(views_file).st_size / 1024 ** 2
    total_size_mb += size_mb
    total_rows += rows
    print(views_file.name, f'{size_mb:,.0f}', f'{rows:,}')
print(total_rows, total_size_mb)


monthly_views_2015.csv 411 9,633,868

monthly_views_2016.csv 659 15,454,988

monthly_views_2019.csv 803 18,844,798

monthly_views_2020.csv 852 20,005,611

monthly_views_2017.csv 708 16,613,180

monthly_views_2022.csv 950 22,310,693

monthly_views_2023.csv 980 23,051,365

monthly_views_2021.csv 904 21,258,947

monthly_views_2018.csv 756 17,731,849

164905299 7023.712878227234

In [12]:
views_df = pl.read_parquet(PROCESSED_DATA_DIR / 'pageviews_all.parquet')
views_df.height * views_df.width

207549090

In [14]:
from scipy import stats
import numpy as np
def regression_per_row(X: np.ndarray):
    # normalize X
    X = (X - X.mean(axis=1, keepdims=True)) / X.std(axis=1, keepdims=True)
    
    # X range (time)
    m = X.shape[1] - 1
    X_range = np.arange(X.shape[1])
    X_range_mean = m / 2
    
    # slopes, intercepts, residuals
    mean = X.mean(axis=1, keepdims=True)
    slopes = ((X - mean).dot(X_range - X_range_mean) / (m * np.var(X_range))).reshape(-1, 1)
    intercepts = mean - slopes * X_range_mean
    residuals = X - (slopes * X_range + intercepts)
    
    # rss, tss, r2
    rss = np.sum(np.square(residuals), axis=1, keepdims=True)
    tss = np.sum(np.square(X - mean), axis=1, keepdims=True)
    r2s = 1 - rss / tss
    
    # pvalues
    se = X.std(axis=1, keepdims=True) / np.sqrt(np.sum(np.square(X_range - X_range_mean)))
    statistics = slopes / se
    pvs = 2 * (1 - stats.t.cdf(np.abs(statistics), m))
    
    return slopes, intercepts, r2s, pvs
    
def is_rising(slopes: np.ndarray, pvs: np.ndarray):
    return np.logical_and(slopes > 0, pvs > 0.05)

def is_fading(slopes: np.ndarray, pvs: np.ndarray):
    return np.logical_and(slopes < 0, pvs > 0.05)

In [16]:
regression_per_row(np.random.randint(1, 10, (1000, 100)))

(array([[ 9.28599153e-04],
        [ 5.97782670e-04],
        [ 3.77025283e-03],
        [-1.12008451e-03],
        [-2.17027418e-04],
        [-7.17646755e-04],
        [ 5.50699921e-03],
        [ 1.28177846e-03],
        [ 1.25064551e-03],
        [-1.79903760e-03],
        [-3.30606667e-03],
        [-1.61399026e-03],
        [ 3.12132147e-03],
        [ 2.42814831e-03],
        [-3.52399538e-03],
        [-3.41078994e-03],
        [-3.25195528e-03],
        [-4.00698134e-03],
        [ 1.66661767e-03],
        [ 1.19245956e-03],
        [-6.84098611e-03],
        [-2.07338669e-03],
        [-1.67854581e-03],
        [-2.72191734e-04],
        [-1.84625078e-03],
        [ 3.26017843e-04],
        [ 2.04692362e-03],
        [-2.95810010e-03],
        [-5.79621697e-03],
        [-1.58673624e-03],
        [ 3.29111663e-03],
        [ 2.58259375e-03],
        [ 2.89965688e-03],
        [ 2.62721372e-03],
        [-6.14481471e-03],
        [-5.30241379e-03],
        [-1.71869367e-03],
 

----

seasonal

In [18]:
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy import stats

def detect_seasonality(data):
    max_period = data.shape[1] // 2  # Maximum possible period
    
    # Create an array to store the p-values
    p_values = np.empty((data.shape[0], max_period))
    
    # Iterate over possible periods and compute p-values for each row
    for period in range(1, max_period + 1):
        result = seasonal_decompose(data, period=period, extrapolate_trend='freq')
        _, p_values[:, period - 1] = stats.normaltest(result.seasonal)
    
    # Check if any row has a significant p-value
    significant_rows = np.any(p_values < 0.05, axis=1)
    
    return significant_rows

# Example usage
data = np.random.rand(10, 100)  # Replace this with your 2D time series data

seasonal_rows = detect_seasonality(data)
print("Indices of rows with seasonality:", np.where(seasonal_rows)[0])


/home/netanel/.conda/envs/big_data_project/lib/python3.12/site-packages/scipy/stats/_stats_py.py:1971: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  k, _ = kurtosistest(a, axis)


ValueError: could not broadcast input array from shape (100,) into shape (10,)

In [19]:
from scipy.stats import kruskal

kruskal([1, 2, 1, 2, 1])

ValueError: Need at least two groups in stats.kruskal()

-----

Notable People

In [10]:
ext = pl.read_csv(RAW_DATA_DIR / 'cross-verified-database.csv',
                  ignore_errors=True)

In [11]:
ext

wikidata_code,birth,death,updated_death_date,approx_birth,approx_death,birth_min,birth_max,death_min,death_max,gender,level1_main_occ,name,un_subregion,birth_estimation,death_estimation,bigperiod_birth_graph_b,bigperiod_death_graph_b,curid,level2_main_occ,freq_main_occ,freq_second_occ,level2_second_occ,level3_main_occ,bigperiod_birth,bigperiod_death,wiki_readers_2015_2018,non_missing_score,total_count_words_b,number_wiki_editions,total_noccur_links_b,sum_visib_ln_5criteria,ranking_visib_5criteria,all_geography_groups,string_citizenship_raw_d,citizenship_1_b,citizenship_2_b,list_areas_of_rattach,area1_of_rattachment,area2_of_rattachment,list_wikipedia_editions,un_region,group_wikipedia_editions,bplo1,dplo1,bpla1,dpla1,pantheon_1,level3_all_occ
str,i64,i64,i64,str,str,i64,i64,i64,i64,str,str,str,str,f64,f64,str,str,i64,str,f64,f64,str,str,str,str,i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,i64,str
"""Q1000002""",1932,1990,null,null,null,1932,1932,1990,1990,"""Male""","""Culture""","""Claus_Hammel""","""Western Europe…",1932.0,1990.0,"""5.Contemporary…","""5.Contemporary…",2949539,"""Culture-core""",0.8,0.2,"""Culture-periph…","""playwright""","""5.Contemporary…","""5.Contemporary…",1669,3,1777,1,11,18.083672,1.058542e6,"""Germany""","""'Germany'""","""Germany""",null,"""D:_'Germany'_m…","""Germany""","""Missing""","""dewiki""","""Europe""","""grB""",11.833333,12.42,53.416668,54.38139,0,"""D:_playwright_…"
"""Q1000005""",1860,1927,null,null,null,1860,1860,1927,1927,"""Male""","""Culture""","""Karel_Matěj_Ča…","""Western Europe…",1860.0,1927.0,"""4.Mid Modern P…","""5.Contemporary…",4217319,"""Culture-core""",0.538462,0.307692,"""Culture-periph…","""writer""","""4.Mid Modern P…","""5.Contemporary…",25008,3,6491,9,15,23.98061,131428.0,"""Czech_Republic…","""'Czech_Republi…","""Czech_Republic…",null,"""D:_'Czech_Repu…","""Old_(before_ye…","""Missing""","""dewiki|cswiki|…","""Europe""","""grA""",12.929798,14.421389,49.440605,50.087502,0,"""D:_writer_jour…"
"""Q1000006""",1971,null,null,null,null,1971,1971,null,null,"""Male""","""Culture""","""Florian_Eichin…","""Western Europe…",1971.0,2053.8447,"""5.Contemporary…","""5.Contemporary…",5050967,"""Culture-core""",1.0,null,"""Missing""","""film""","""5.Contemporary…","""Missing""",27285,3,1573,1,10,20.666656,775768.0,"""Germany""","""'Germany'""","""Germany""",null,"""D:_'Germany'_m…","""Germany""","""Missing""","""dewiki""","""Europe""","""grB""",9.191944,null,48.897499,null,0,"""D:_film_screen…"
"""Q1000015""",1983,null,null,null,null,1983,1983,null,null,"""Male""","""Culture""","""Florian_Jahr""","""Western Europe…",1983.0,2067.1899,"""5.Contemporary…","""5.Contemporary…",2588583,"""Culture-core""",1.0,null,"""Missing""","""actor""","""5.Contemporary…","""Missing""",37331,3,1931,1,10,21.18504,691735.0,"""Germany""","""'Germany'""","""Germany""",null,"""D:_'Germany'_m…","""Germany""","""Missing""","""dewiki""","""Europe""","""grB""",13.383333,null,52.516666,null,0,"""D:_actor_P:_sc…"
"""Q1000023""",1912,1977,null,null,null,1912,1912,1977,1977,"""Female""","""Leadership""","""Wiltraut_Rupp-…","""Western Europe…",1912.0,1977.0,"""5.Contemporary…","""5.Contemporary…",922120,"""Administration…",0.833333,0.166667,"""Politics""","""judge""","""5.Contemporary…","""5.Contemporary…",2955,3,1578,1,6,17.99621,1.103282e6,"""Germany""","""'Germany'""","""Germany""",null,"""D:_'Germany'_m…","""Old_(before_ye…","""Missing""","""dewiki""","""Europe""","""grB""",13.35,8.4,52.4333,49.016666,0,"""D:_judge_juris…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Q999994""",1988,null,null,null,null,1988,1988,null,null,"""Male""","""Sports/Games""","""Mitja_Mežnar""","""Southern Europ…",1988.0,2065.3259,"""5.Contemporary…","""5.Contemporary…",20581328,"""Sports/Games""",0.9,null,"""Missing""","""ski""","""5.Contemporary…","""Missing""",8896,3,5668,8,4,21.543238,350041.5,"""Slovenia""","""'Slovenia'""","""Slovenia"""

- `name`: full name of the individual; 
- `group_wikipedia_editions`: partition category of the individual (from the A (*English*) edition of Wikipedia to F (\texttt{Wikidata} only) as described in Table 1.); 
- birth: birth date of the individual (either reported or estimated); 
- death: death date of the individual (either reported or estimated); 
- level1_main_occ & level2_main_occ & level2_second_occ & level3_all_occ & level3_main_occ & freq_main_ occ & freq_second_occ: set of seven variables for the main domain of influence of each individual in three

- `wikidata_code` - identifier
- `birth` - birth year of the individual
- `death` - death year of the individual
- `updated_death_date`
- `approx_birth`
- `approx_death`
- `birth_min`
- `birth_max`
- `death_min`
- `death_max`
- `gender`
- `level1_main_occ`
- `name`
- `un_subregion`
- `birth_estimation`
- `death_estimation`
- `bigperiod_birth_graph_b`
- `bigperiod_death_graph_b`
- `curid`
- `level2_main_occ`
- `freq_main_occ`
- `freq_second_occ`
- `level2_second_occ`
- `level3_main_occ`
- `bigperiod_birth`
- `bigperiod_death`
- `wiki_readers_2015_2018`
- `non_missing_score`
- `total_count_words_b`
- `number_wiki_editions`
- `total_noccur_links_b`
- `sum_visib_ln_5criteria`
- `ranking_visib_5criteria`
- `all_geography_groups`
- `string_citizenship_raw_d`
- `citizenship_1_b`
- `citizenship_2_b`
- `list_areas_of_rattach`
- `area1_of_rattachment`
- `area2_of_rattachment`
- `list_wikipedia_editions`
- `un_region`
- `group_wikipedia_editions`
- `bplo1`
- `dplo1`
- `bpla1`
- `dpla1`
- `pantheon_1`
- `level3_all_occ`

In [18]:
for col in ext:
    display(col.value_counts(sort=True))

wikidata_code,count
str,u32
"""Q1000002""",1
"""Q1000005""",1
"""Q1000006""",1
"""Q1000015""",1
"""Q1000023""",1
…,…
"""Q999994""",1
"""Q999995""",1
"""Q999997""",1


birth,count
i64,u32
null,195919
1950,20227
1988,20215
1985,20134
1986,19998
…,…
-599,1
-274,1
-1440,1


death,count
i64,u32
null,1244507
2015,12406
2013,12352
2014,12308
2016,12039
…,…
-452,1
-540,1
-1415,1


updated_death_date,count
i64,u32
null,2275077
2021,10321
2022,4194
2020,2225


approx_birth,count
str,u32
null,2201780
"""century""",52613
"""circa""",37390
"""millenium""",34


...

dplo1,count
f64,u32
null,1614372
2.351389,21586
12.482778,12439
13.383333,11784
-0.1275,10120
…,…
9.05611,1
86.7715,1
-0.327222,1


bpla1,count
f64,u32
null,587627
48.856945,17385
52.516666,16240
40.700001,15232
51.507221,14229
…,…
48.8769,1
51.8839,1
44.605278,1


dpla1,count
f64,u32
null,1614372
48.856945,21490
41.893055,12454
52.516666,11717
51.507221,10130
…,…
52.339699,1
48.938599,1
54.671299,1


pantheon_1,count
i64,u32
0,2280027
1,11790


level3_all_occ,count
str,u32
"""D:_football_P:…",32329
"""D:_P:""",15417
"""D:_politician_…",13686
"""D:_football_P:…",12250
"""D:_cricket_P:_…",11295
…,…
"""D:_ski_P:_ ski…",1
"""D:_ski_P:_ ski…",1
"""D:_ski_skier_P…",1
